<a href="https://colab.research.google.com/github/mao1756/CAP5619-sp24/blob/main/q3main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Q3 Notebook

In [12]:
!wget http://www.cs.fsu.edu/~liux/courses/deepRL/assignments/hw2_softmax_weights.m
!pip install oct2py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 1.4 MB/s eta 0:00:00


In [13]:
# save weights
from oct2py import octave as oct
oct.eval('run hw2_softmax_weights.m')
oct.eval("save -v7 weights.mat")

In [18]:
# load weights using scipy
import scipy.io as sio
weights = sio.loadmat('weights.mat')
bias = weights['softmax_bias']
weights = weights['softmax_weight']

#Q1

In [23]:
# check the shapes
print(bias.shape)
print(weights.shape)

(1, 20)
(100, 20)


# Q2

In [26]:
# import the samples
!wget http://www.cs.fsu.edu/~liux/courses/deepRL/assignments/hw2_softmax_sample.txt
oct.eval('run hw2_softmax_sample.txt')
oct.eval("save -v7 sample.mat")
sam = sio.loadmat('sample.mat')
samples = sam['asample']

--2024-02-06 21:18:39--  http://www.cs.fsu.edu/~liux/courses/deepRL/assignments/hw2_softmax_sample.txt
Resolving www.cs.fsu.edu (www.cs.fsu.edu)... 128.186.122.19
Connecting to www.cs.fsu.edu (www.cs.fsu.edu)|128.186.122.19|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 662 [text/plain]
Saving to: ‘hw2_softmax_sample.txt.2’

hw2_softmax_sample. 100%[===================>]     662  --.-KB/s    in 0s      

2024-02-06 21:18:39 (82.3 MB/s) - ‘hw2_softmax_sample.txt.2’ saved [662/662]



In [37]:
samples = samples.reshape(-1)

In [70]:
import torch

class Softmax(torch.nn.Module):
  def __init__(self, weights, bias):
    super(Softmax, self).__init__()
    self.linear = torch.nn.Linear(weights.shape[0], weights.shape[1])
    self.linear.weight = torch.nn.Parameter(torch.tensor(weights.T))
    self.linear.bias = torch.nn.Parameter(torch.tensor(bias.flatten()))
    self.softmax = torch.nn.Softmax(dim=0)

  def forward(self, x):
    x = self.linear(x)
    x = self.softmax(x)
    return x

# define the model
model_softmax = Softmax(weights, bias)

# forward pass
outputs = model_softmax(torch.tensor(samples))

# classify the sample
predictions = torch.argmax(outputs)
predictions = predictions.numpy()
print(predictions)

3


#Q3

In [68]:
opt = torch.optim.SGD(model_softmax.parameters(), lr=0.1)
opt.zero_grad()
loss = -torch.log(outputs[5]) # cross_entopy
loss.backward()
opt.step()


# check how many weights and biases incread/decreased/same
# increased weights
print((model_softmax.linear.weight-torch.tensor(weights.T)>0.00001).sum())
# decreased weights
print((model_softmax.linear.weight-torch.tensor(weights.T)<-0.00001).sum())
# same weights
print(((model_softmax.linear.weight-torch.tensor(weights.T)<0.00001)
& (model_softmax.linear.weight-torch.tensor(weights.T)>-0.00001)).sum())

# increased bias
print(sum(model_softmax.linear.bias-torch.tensor(bias.flatten())>0.00001))
# decreased bias
print(sum(model_softmax.linear.bias-torch.tensor(bias.flatten())<-0.00001))
# same weights
print(sum((model_softmax.linear.bias-torch.tensor(bias.flatten())<0.00001 ) & (model_softmax.linear.bias-torch.tensor(bias.flatten())>-0.00001)))


tensor(10)
tensor(180)
tensor(1810)
tensor(1)
tensor(18)
tensor(1)


# Q4

In [ ]:
from scipy.optimize import fmin_l_bfgs_b

